In [2]:
import numpy as np
import jdc
import random
import matplotlib.pyplot as plt

In [3]:
class Road_Alignment:
    def __init__(self,data,pop_size):
        self.data = data
        self.shape = data.shape
        self.pop_size = pop_size
        self.population = None

In [4]:
%%add_to Road_Alignment
def initialize_population(self):
    self.population = []
    for i in range(self.pop_size):
        chromosome = self._initialize_chromosome()
        self.population.append(chromosome)

In [5]:
%%add_to Road_Alignment
def _initialize_chromosome(self):
    chromosome = [(-1, -1)] * (self.data.shape[1])
    min_genes = 1
    max_genes = self.data.shape[1]
#     max_genes = 10
    num_genes = random.randint(min_genes, max_genes)
    genes = self._initialize_genes(num_genes)
    chromosome[0] = genes[0] 
    chromosome[-1] = genes[-1] 
    last_index = 0
    for idx, (i, j) in enumerate(genes):
        if idx == 0 or idx == len(genes) -1:
            continue
        next_index = random.randint(last_index + 1, len(chromosome) - len(genes) + idx)
        chromosome[next_index] = (i,j)
        last_index = next_index
    return chromosome

In [6]:
%%add_to Road_Alignment
def _initialize_genes(self, num_genes):
    first_gene = (0, random.randint(0, self.data.shape[0] - 1))
    distinct_tuples = []
    distinct_tuples.append(first_gene)
    count = 0
    while len(distinct_tuples) < num_genes:
        new_tuple = self._initialize_gene()
        if new_tuple[0] == self.data.shape[1] - 1:
            break
        if new_tuple[0] > distinct_tuples[-1][0]:
            distinct_tuples.append(new_tuple)
    if distinct_tuples[-1] != (self.data.shape[1] - 1):
        y_coordinate = random.randint(0,self.data.shape[0] - 1)
        distinct_tuples.append((self.data.shape[1] - 1 ,y_coordinate))
    return distinct_tuples

In [7]:
%%add_to Road_Alignment
def _initialize_gene(self):
    gene = random.randint(0, 59), random.randint(0, 19)
    return gene

In [8]:
%%add_to Road_Alignment
def _find_coordinates(self,chromosome,value):
    indices = np.where(chromosome == value)
    x, y = indices[0][0], indices[1][0]
    return x,y

In [9]:
%%add_to Road_Alignment
def chromosome_genes(self,chromosome):
    genes = []
    for i, (x,y) in enumerate(chromosome):
        if (x,y) != (-1, -1):
            genes.append((x,y))
    return genes

In [10]:
%%add_to Road_Alignment
def calculate_fitness(self,chromosome):
    
    genes = self.chromosome_genes(chromosome)
    total_cost = 0.0 
    for i in range(len(genes) - 1):
        x1, y1 = genes[i]
        x2, y2 = genes[i + 1]
        if abs(x1 - x2) == 1 and abs(y1 - y2) <= 1:
            road_cost = self.data[y1][x1]
            total_cost += road_cost
        else:
            distance_cost = (x2 - x1)**2 + (y2 - y1)**2
            total_cost += distance_cost

    return total_cost

In [11]:
%%add_to Road_Alignment
def parent_tournament_selection(self, tournament_size):
    tournament_candidates = random.sample(self.population, tournament_size)
    winner = min(tournament_candidates, key=lambda ind: self.calculate_fitness(ind))
    tournament_candidates = random.sample(self.population, tournament_size)
    runner_up = min(tournament_candidates, key=lambda ind: self.calculate_fitness(ind))
    return winner, runner_up

In [12]:
%%add_to Road_Alignment
def uniform_crossover(self,parent1, parent2, crossover_probability=0.5):
    offspring1 = np.zeros_like(parent1)
    offspring2 = np.zeros_like(parent2)

    for i in range(len(parent1)):
        if np.random.rand() < crossover_probability:
            offspring1[i] = parent1[i]
            offspring2[i] = parent2[i]
        else:
            offspring1[i] = parent2[i]
            offspring2[i] = parent1[i]

    return offspring1, offspring2

In [13]:
%%add_to Road_Alignment
def mutate(self,chromosome,mutation_probability = 0.1):
    mutated_chromosome = np.copy(chromosome)

    for i in range(len(chromosome)):
            
        if np.random.rand() < mutation_probability:
            x, y = mutated_chromosome[i]
            if i == 0 or i == len(chromosome) - 1:
                y = np.clip(y + np.random.randint(-2, 2), 0, self.data.shape[0] - 1)
                mutated_chromosome[i] = (x, y)
                continue
            if np.random.rand() < 0.5:
                x = np.clip(x + np.random.randint(-5, 5), 0, self.data.shape[1] - 1)
                y = np.clip(y + np.random.randint(-2, 2), 0, self.data.shape[0] - 1)
            else:
                x = -1
                y = -1
            mutated_chromosome[i] = (x, y)

    return list(mutated_chromosome)

In [14]:
%%add_to Road_Alignment
def terminate(self, generation, max_generations, convergence_threshold, no_improvement_threshold, best_fitness_values):
    if generation >= max_generations:
        return True
    
    if has_converged(best_fitness_values, convergence_threshold):
        return True
    
    if no_improvement(best_fitness_values, no_improvement_threshold):
        return True
    
    return False

In [15]:
def has_converged(best_fitness_values, convergence_threshold):
    if len(best_fitness_values) > 1:
        last_best_fitness = best_fitness_values[-2]
        current_best_fitness = best_fitness_values[-1]
        
        change_in_fitness = np.abs(current_best_fitness - last_best_fitness)
        
        return change_in_fitness < convergence_threshold
    
    return False

In [16]:
def no_improvement(best_fitness_values, no_improvement_threshold):
    if len(best_fitness_values) > no_improvement_threshold:
        recent_best_fitness = best_fitness_values[-no_improvement_threshold:]
        improvement_check = all(recent_best_fitness[i] >= recent_best_fitness[i + 1] for i in range(no_improvement_threshold - 1))   
        return improvement_check
    
    return False
    

In [17]:
%%add_to Road_Alignment
def extract_solution(self,population, fitness_values):
    best_index = np.argmin(fitness_values)
    best_individual = population[best_index]
    
    return best_individual

In [18]:
%%add_to Road_Alignment
def visualize_map(self):
    plt.imshow(self.data, cmap='YlGn', interpolation='nearest')
    cbar = plt.colorbar()
    plt.gca().invert_yaxis()
    cbar.set_label('Roughness Level')
    plt.title('Map Roughness')
#     plt.show()
    return plt

In [19]:
%%add_to Road_Alignment
def visualize_chromosome(self, plt,chromosome):
    genes = self.chromosome_genes(chromosome)
    for i in range(len(genes) - 1):
        x1, y1 = genes[i]
        x2, y2 = genes[i + 1]
        if abs(x1 - x2) == 1 and abs(y1 - y2) <= 1:
            plt.plot([x1, x2], [y1, y2], marker='o', linestyle='-',color='black')
        else:
            plt.plot([x1, x2], [y1, y2], marker='o', linestyle='-',color='red')

    plt.title('chromosome')
    plt.show()

In [20]:
data = np.load('map.npy')
ra = Road_Alignment(data,100)
ra.initialize_population()

In [ ]:
max_generations = 300
tournament_size = 5
crossover_probability = 0.7
mutation_probability = 0.1
convergence_threshold = 1e-6
no_improvement_threshold = 10

best_fitness_values = []
count = 0
for generation in range(max_generations):
    # Evaluate fitness for each individual in the population
    fitness_values = [ra.calculate_fitness(ind) for ind in ra.population]

    # Record the best fitness value for visualization
    best_fitness_values.append(min(fitness_values))

    # Select parents using tournament selection
    parent1, parent2 = ra.parent_tournament_selection(tournament_size)

    # Apply crossover to create offspring
    offspring1, offspring2 = ra.uniform_crossover(parent1, parent2, crossover_probability)

    # Apply mutation to the offspring
    mutated_offspring1 = ra.mutate(offspring1, mutation_probability)
    mutated_offspring2 = ra.mutate(offspring2, mutation_probability)

    # Replace the two worst individuals in the population with the mutated offspring
    ra.population[fitness_values.index(max(fitness_values))] = mutated_offspring1
    ra.population[fitness_values.index(max(fitness_values))] = mutated_offspring2


    # Check for termination conditions
#     if ra.terminate(generation, max_generations, convergence_threshold, no_improvement_threshold, best_fitness_values):
#         break
    # Visualize the best individual in the current generation
    
best_individual = ra.extract_solution(ra.population, fitness_values)
print(ra.calculate_fitness(best_individual))
plt = ra.visualize_map()
ra.visualize_chromosome(plt, best_individual)


In [446]:
data.shape[0] - 1

19